<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/drbob2142_mix_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>DrBob2142's Mix Models Colab</code></h2>

- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab#-how-to-use)
- 🤗 [HuggingFace Repo](https://huggingface.co/DrBob2142/Mix-Models)

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown Select a model before running
model = "Midnight Mix Alt.safetensors" #@param ["Blossom-extract.safetensors", "Docs 50 - 50 V2.safetensors", "Midnight Mix Alt.safetensors", "Midnight Mix.safetensors", "Midnight Mixer Alt V2 .safetensors", "Midnight Mixer V2 .safetensors", "Midnight Shake .safetensors", "Yohan Bake V1.safetensors", "Yohan Potluck V1.safetensors", "Yohan Potluck V2.safetensors", "Yohan Spiced V1.safetensors", "Yohan Spiced V2.safetensors"]

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.app` link to the web UI

try:
  has_run
except NameError:
  has_run = False

huggingface_repo = "https://huggingface.co/DrBob2142/Mix-Models"

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"

git_flags = "-q"
wget_flags = "-q --show-progress"

vae_link = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"
vae_name = vae_link.split("/")[-1]

if not has_run:
  has_run = True
  
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
  
  !git clone {git_flags} https://github.com/camenduru/stable-diffusion-webui
  !git clone {git_flags} https://github.com/yfszzx/stable-diffusion-webui-images-browser {extensions_folder}/stable-diffusion-webui-images-browser
  !git clone {git_flags} https://github.com/DominikDoom/a1111-sd-webui-tagcomplete {extensions_folder}/a1111-sd-webui-tagcomplete
  
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/anything_ui_config.json -O {web_ui_folder}/ui-config.json
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/config.json -O {web_ui_folder}/config.json
  !wget {wget_flags} https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/styles.csv -O {web_ui_folder}/styles.csv
  
  !wget {wget_flags} {vae_link} -O {vae_folder}/{vae_name}

try:
  models_downloaded
except NameError:
  models_downloaded = []

if model not in models_downloaded:
  models_downloaded.append(model)
  !wget {wget_flags} "{huggingface_repo}/resolve/main/{model}" -O "{models_folder}/{model}"

%cd {web_ui_folder}

arg_list = [
  "--share --xformers",
  f"--vae-path {vae_folder}/{vae_name}",
  f"--ckpt \"{models_folder}/{model}\"",
]

args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 📩 `Saving images`

#@markdown The zip file will be visible at the files tab.

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"Zipped. You can now find {archive_name} at the files tab.")


#@markdown 📤 Upload zip to [temp.sh](https://temp.sh/)?
upload_to_tempsh = False #@param {type:'boolean'}
if upload_to_tempsh:
  !curl -T /content/{archive_name} temp.sh
  print("\n^^^ This is your download link! ^^^")

#@markdown ----

#@markdown 📲 Copy zip to Google Drive?
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"Copied to {gdrive_folder}!")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = True #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs